In [1]:
import pandas as pd

In [2]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [3]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [4]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [5]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  1072.0


In [6]:
# Confirmados PCR Huelva capital
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  285.0


In [7]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [8]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2612 personas en los últimos 7 días 

Un positivo PCR cada 1890 personas en los últimos 14 días


In [9]:
filtro = ((df['Medida'] == 'Confirmados PCR')| (df['Medida'] == 'Confirmados PCR 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PCR 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [10]:
# Reorder columns
tabla = tabla[['Confirmados PCR', 'Confirmados PCR 7 días', 'Confirmados PCR 14 días', 'Población']]

In [11]:
tabla['tasa7'] = tabla['Confirmados PCR 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PCR 14 días']/tabla['Población']*100000

In [12]:
tabla['Diferencia PCR'] = tabla['Confirmados PCR'] - dfAnterior['Confirmados PCR']

In [13]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PCR'] > 0].sort_values('Diferencia PCR', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Huelva,1072.0,181.0,333.0,510743.0,35.438567,65.199131,32.0
Huelva-Costa,577.0,125.0,188.0,288115.0,43.385454,65.251722,17.0
Condado-Campiña,319.0,38.0,110.0,155057.0,24.507117,70.941654,13.0
Huelva (capital),285.0,55.0,76.0,143663.0,38.284040,52.901582,9.0
Bollullos Par del Condado,40.0,9.0,26.0,14272.0,63.060538,182.174888,5.0
Ayamonte,34.0,17.0,22.0,20946.0,81.161081,105.031987,3.0
Nerva,18.0,4.0,4.0,5235.0,76.408787,76.408787,3.0
Sierra de Huelva-Andévalo Central,157.0,16.0,24.0,67571.0,23.678797,35.518196,3.0
Villalba del Alcor,41.0,6.0,27.0,3338.0,179.748352,808.867585,3.0


In [14]:
if tabla[tabla['Diferencia PCR'] > 0]['Diferencia PCR'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [15]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Cabezas Rubias,9.0,6.0,8.0,730.0,821.917808,1095.890411,1.0
Villalba del Alcor,41.0,6.0,27.0,3338.0,179.748352,808.867585,3.0
Zalamea la Real,19.0,4.0,6.0,3068.0,130.378096,195.567145,0.0
Bollullos Par del Condado,40.0,9.0,26.0,14272.0,63.060538,182.174888,5.0
Isla Cristina,52.0,14.0,31.0,21264.0,65.838977,145.786305,0.0
Palos de la Frontera,22.0,11.0,14.0,11289.0,97.439986,124.014527,1.0
Cortegana,9.0,1.0,5.0,4666.0,21.431633,107.158165,0.0
Ayamonte,34.0,17.0,22.0,20946.0,81.161081,105.031987,3.0
Aljaraque,65.0,17.0,22.0,21260.0,79.962371,103.480715,2.0
